In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
import numpy as np
from math import sqrt, pi, log, e
from enum import Enum
import scipy.stats as stat
from scipy.stats import norm
import time



In [7]:
class BSMerton(object):
    def __init__(self, args):
        self.Type = int(args[0])  # 1 for a Call, - 1 for a put
        self.S = float(args[1])  # Underlying asset price
        self.K = float(args[2])  # Option strike K
        self.r = float(args[3])  # Continuous risk fee rate
        self.q = float(args[4])  # Dividend continuous rate
        self.T = float(args[5]) / 365.0  # Compute time to expiry
        self.sigma = float(args[6])  # Underlying volatility
        self.sigmaT = self.sigma * self.T ** 0.5  # sigma*T for reusability
        self.d1 = (log(self.S / self.K) + \
                   (self.r - self.q + 0.5 * (self.sigma ** 2)) \
                   * self.T) / self.sigmaT
        self.d2 = self.d1 - self.sigmaT
        [self.Premium] = self.premium()
        [self.Delta] = self.delta()
        [self.Theta] = self.theta()
        [self.Rho] = self.rho()
        [self.Vega] = self.vega()
        [self.Gamma] = self.gamma()
        [self.Phi] = self.phi()
        [self.Charm] = self.dDeltadTime()
        [self.Vanna] = self.dDeltadVol()

    def premium(self):
        tmpprem = self.Type * (self.S * e ** (-self.q * self.T) * norm.cdf(self.Type * self.d1) - \
                               self.K * e ** (-self.r * self.T) * norm.cdf(self.Type * self.d2))
        return [tmpprem]

    ############################################
    ############ 1st order greeks ##############
    ############################################

    def delta(self):
        dfq = e ** (-self.q * self.T)
        if self.Type == 1:
            return [dfq * norm.cdf(self.d1)]
        else:
            return [dfq * (norm.cdf(self.d1) - 1)]

    # Vega for 1% change in vol
    def vega(self):
        return [0.01 * self.S * e ** (-self.q * self.T) * \
                norm.pdf(self.d1) * self.T ** 0.5]

    # Theta for 1 day change
    def theta(self):
        df = e ** -(self.r * self.T)
        dfq = e ** (-self.q * self.T)
        tmptheta = (1.0 / 365.0) \
                   * (-0.5 * self.S * dfq * norm.pdf(self.d1) * \
                      self.sigma / (self.T ** 0.5) + \
                      self.Type * (self.q * self.S * dfq * norm.cdf(self.Type * self.d1) \
                                   - self.r * self.K * df * norm.cdf(self.Type * self.d2)))
        return [tmptheta]

    def rho(self):
        df = e ** -(self.r * self.T)
        return [self.Type * self.K * self.T * df * 0.01 * norm.cdf(self.Type * self.d2)]

    def phi(self):
        return [0.01 * -self.Type * self.T * self.S * \
                e ** (-self.q * self.T) * norm.cdf(self.Type * self.d1)]

    ############################################
    ############ 2nd order greeks ##############
    ############################################

    def gamma(self):
        return [e ** (-self.q * self.T) * norm.pdf(self.d1) / (self.S * self.sigmaT)]

    # Charm for 1 day change
    def dDeltadTime(self):
        dfq = e ** (-self.q * self.T)
        if self.Type == 1:
            return [
                (1.0 / 365.0) * -dfq * (norm.pdf(self.d1) * ((self.r - self.q) / (self.sigmaT) - self.d2 / (2 * self.T)) \
                                        + (-self.q) * norm.cdf(self.d1))]
        else:
            return [
                (1.0 / 365.0) * -dfq * (norm.pdf(self.d1) * ((self.r - self.q) / (self.sigmaT) - self.d2 / (2 * self.T)) \
                                        + self.q * norm.cdf(-self.d1))]

    # Vanna for 1% change in vol
    def dDeltadVol(self):
        return [0.01 * -e ** (-self.q * self.T) * self.d2 / self.sigma * norm.pdf(self.d1)]

    # Vomma
    def dVegadVol(self):
        return [0.01 * -e ** (-self.q * self.T) * self.d2 / self.sigma * norm.pdf(self.d1)]

In [21]:
class BSMertonImpledVolFinder(object):
    def __init__(self, args):
        self.Type = int(args[0])  # 1 for a Call, - 1 for a put
        self.S = float(args[1])  # Underlying asset price
        self.K = float(args[2])  # Option strike K
        self.r = float(args[3])  # Continuous risk fee rate
        self.q = float(args[4])  # Dividend continuous rate
        self.T = float(args[5]) / 365.0  # Compute time to expiry
        self.Premium = float(args[6])  # Option premium
        self.sigma = self.implied_volatility()

    def black_scholes_price(self, sigma):
        sigmaT = sigma * self.T ** 0.5
        d1 = (log(self.S / self.K) + (self.r - self.q + 0.5 * (sigma ** 2)) * self.T) / sigmaT
        d2 = d1 - sigmaT
        price = self.Type * (self.S * e ** (-self.q * self.T) * norm.cdf(self.Type * d1) - \
                             self.K * e ** (-self.r * self.T) * norm.cdf(self.Type * d2))
        return price

    def implied_volatility(self, epsilon=0.0001):
        # Use the bisection method to find the implied volatility that matches the given option premium
        lower = 0.001
        upper = 10.0
        mid = (lower + upper) / 2.0
        diff = self.Premium - self.black_scholes_price(mid)
        while abs(diff) > epsilon:
            if diff > 0:
                lower = mid
            else:
                upper = mid
            mid = (lower + upper) / 2.0
            diff = self.Premium - self.black_scholes_price(mid)
        return mid
    

In [20]:
# Call = 1  Put = -1   [Option,Price, Strike,Continuous rf rate, dividend, days to exp,vol]

Implied_Volatility = [.1,.15,.2,.25,.3,.35,.4,.45,.5,.55,.6,65,.7,.75,.8]

AAPL_Call= [1,165,165,0.0425,0.0053,14,Implied_Volatility]
AAPL_Put= [-1,165,165,0.0425,0.0053,14,Implied_Volatility]


def ArrayOfPricesGivenVols(OptionParams,IV_Array):
    prices = []
    for i in IV_Array:
        OptionParams[6] = i
        prices.append(BSMerton(OptionParams).Premium)
    return prices

AAPLCallSide= ArrayOfPricesGivenVols(AAPL_Call,Implied_Volatility)
AAPLPutSide= ArrayOfPricesGivenVols(AAPL_Put,Implied_Volatility)

AAPLCallSide
AAPLPutSide

[1.173782017848609,
 1.8165913138241478,
 2.4599232051813402,
 3.103421072678273,
 3.746955662812084,
 4.390462753433198,
 5.033902524676748,
 5.677246009596956,
 6.320469673507574,
 6.9635529502028675,
 7.606477010470343,
 8.24922409943423,
 8.891777158342094,
 9.534119598064947,
 10.176235157940582]

In [14]:
AAPL_Call_Sanity_Check = [1,165,165,0.0425,0.0053,14,.8]
BSMerton(AAPL_Call_Sanity_Check).Premium



10.411449592063775

In [24]:
AAPL_IV_Call_150 = [1,151.03,155,0.0425,0.0053,14,4.05]

ImpVol = BSMertonImpledVolFinder(AAPL_IV_Call_150).implied_volatility()
ImpVol


0.47994456291198717